# 🤗 Welcome to AdalFlow!
## The PyTorch library to auto-optimize any LLM task pipelines

Thanks for trying us out, we're here to provide you with the best LLM application development experience you can dream of 😊 any questions or concerns you may have, [come talk to us on discord,](https://discord.gg/ezzszrRZvT) we're always here to help!


# Quick Links

Github repo: https://github.com/SylphAI-Inc/AdalFlow

Full Tutorials: https://adalflow.sylph.ai/index.html#.

Deep dive on each API: check out the [developer notes](https://adalflow.sylph.ai/tutorials/index.html).

Common use cases along with the auto-optimization:  check out [Use cases](https://adalflow.sylph.ai/use_cases/index.html).

# Outline

*Note: As training can consume tokens fast, and the notebook runtime will reset everytime you use, it might be better for you to learn training in your local editor.*

This is a quick introduction of AdalFlow on question answering use case end to end

* Trainable Task pipeline with trainable parameters
* Create AdalComponent for your task pipeline
* Use Trainer to diagnose, debug, and to train.

You can find all source code here: https://github.com/SylphAI-Inc/AdalFlow/tree/main/use_cases/question_answering/bhh_object_count

**Here is the more detailed tutorial for the code here: https://adalflow.sylph.ai/use_cases/question_answering.html**


# Installation

1. Use `pip` to install the `adalflow` Python package. We will need `openai`, `groq`, and `faiss`(cpu version) from the extra packages.

  ```bash
  pip install adalflow[openai,groq,faiss-cpu]
  ```
2. Setup  `openai` and `groq` API key in the environment variables

In [ ]:
from IPython.display import clear_output

!pip install -U adalflow[openai,groq,datasets]

clear_output()

In [ ]:
import adalflow as adal

adal.__version__

'0.2.0'

## Set Environment Variables

Run the following code and pass your api key.

Note: for normal `.py` projects, follow our [official installation guide](https://lightrag.sylph.ai/get_started/installation.html).

*Go to [OpenAI](https://platform.openai.com/docs/introduction) and [Groq](https://console.groq.com/docs/) to get API keys if you don't already have.*

In [ ]:
import os

from getpass import getpass

# Prompt user to enter their API keys securely
openai_api_key = getpass("Please enter your OpenAI API key: ")
groq_api_key = getpass("Please enter your GROQ API key: ")


# Set environment variables
os.environ['OPENAI_API_KEY'] = openai_api_key
os.environ['GROQ_API_KEY'] = groq_api_key

print("API keys have been set.")

Please enter your OpenAI API key: ··········
Please enter your GROQ API key: ··········
API keys have been set.




# 😇 Trainable Task Pipeline

We will create a task pipeline consists of a generator, with a customzied template, a customized output parser.

Different from our other pipelines where the `prompt_kwargs` values are strings, but here we will use ``Parameter``. And we will set up two parameter, one is of ``ParameterType.PROMPT`` and the other of type ``ParameterType.DEMOS``. The first one will be trained by text-grad and the second will be trained by boostrap few shot optimizer.




In [ ]:
import adalflow as adal
import re
from typing import Dict, Union
import adalflow as adal
from adalflow.optim.types import ParameterType


@adal.fun_to_component
def parse_integer_answer(answer: str):
    """A function that parses the last integer from a string using regular expressions."""
    try:
        # Use regular expression to find all sequences of digits
        numbers = re.findall(r"\d+", answer)
        if numbers:
            # Get the last number found
            answer = int(numbers[-1])
        else:
            answer = -1
    except ValueError:
        answer = -1

    return answer


few_shot_template = r"""<START_OF_SYSTEM_PROMPT>
{{system_prompt}}
{# Few shot demos #}
{% if few_shot_demos is not none %}
Here are some examples:
{{few_shot_demos}}
{% endif %}
<END_OF_SYSTEM_PROMPT>
<START_OF_USER>
{{input_str}}
<END_OF_USER>
"""

class ObjectCountTaskPipeline(adal.Component):
    def __init__(self, model_client: adal.ModelClient, model_kwargs: Dict):
        super().__init__()

        system_prompt = adal.Parameter(
            data="You will answer a reasoning question. Think step by step. The last line of your response should be of the following format: 'Answer: $VALUE' where VALUE is a numerical value.",
            role_desc="To give task instruction to the language model in the system prompt",
            requires_opt=True,
            param_type=ParameterType.PROMPT,
        )
        few_shot_demos = adal.Parameter(
            data=None,
            role_desc="To provide few shot demos to the language model",
            requires_opt=True,  # Changed to True for few-shot learning
            param_type=ParameterType.DEMOS,
        )

        self.llm_counter = adal.Generator(
            model_client=model_client,
            model_kwargs=model_kwargs,
            template=few_shot_template,
            prompt_kwargs={
                "system_prompt": system_prompt,
                "few_shot_demos": few_shot_demos,
            },
            output_processors=parse_integer_answer,
            use_cache=True,
        )

    def call(
        self, question: str, id: str = None
    ) -> Union[adal.GeneratorOutput, adal.Parameter]:
        output = self.llm_counter(prompt_kwargs={"input_str": question}, id=id)
        return output




Next, we will run this pipeline in both train and eval mode.

#### Eval mode with GeneratorOutput

Eval mode will output ``GeneratorOutput``.




#### Train mode with different form of output

Train mode will return ``Parameter``, where the `data` field will be the `raw_response`` from the GeneratorOutput, and we put the full GeneratorOutput at the ``full_response`` in the parameter.

As the `data` field of the `Parameter` directly communicate with the Optimizer, which are an LLM itself, its better than they understand exactly the string response itself instead of the parsed one.

Later you will see that we also use ``eval_input`` of the parameter to communicate with the `LossFunction` as that need the parsed final output.

In [ ]:
from adalflow.components.model_client.openai_client import OpenAIClient
from adalflow.components.model_client.groq_client import GroqAPIClient


gpt_3_model = {
    "model_client": OpenAIClient(),
    "model_kwargs": {
        "model": "gpt-3.5-turbo",
        "max_tokens": 2000,
        "temperature": 0.0,
        "top_p": 0.99,
        "frequency_penalty": 0,
        "presence_penalty": 0,
        "stop": None,
    },
}

llama_3_1_model ={
    "model_client": GroqAPIClient(),
    "model_kwargs": {
        "model": "llama-3.1-8b-instant"
    }
}

gpt_4o_model = {
    "model_client": OpenAIClient(),
    "model_kwargs": {
        "model": "gpt-4o",
        "max_tokens": 4000,
        "temperature": 0.0,
        "top_p": 0.99,
        "frequency_penalty": 0,
        "presence_penalty": 0,
        "stop": None,
    },
}


question = "I have a flute, a piano, a trombone, four stoves, a violin, an accordion, a clarinet, a drum, two lamps, and a trumpet. How many musical instruments do I have?"
task_pipeline = ObjectCountTaskPipeline(**gpt_3_model)
print(task_pipeline)


cache_path: /root/.adalflow/cache_OpenAIClient_gpt-3.5-turbo.db
ObjectCountTaskPipeline(
  (llm_counter): Generator(
    model_kwargs={'model': 'gpt-3.5-turbo', 'max_tokens': 2000, 'temperature': 0.0, 'top_p': 0.99, 'frequency_penalty': 0, 'presence_penalty': 0, 'stop': None}, 
    (prompt): Prompt(
      template: <START_OF_SYSTEM_PROMPT>
      {{system_prompt}}
      {# Few shot demos #}
      {% if few_shot_demos is not none %}
      Here are some examples:
      {{few_shot_demos}}
      {% endif %}
      <END_OF_SYSTEM_PROMPT>
      <START_OF_USER>
      {{input_str}}
      <END_OF_USER>
      , prompt_kwargs: {'system_prompt': "You will answer a reasoning question. Think step by step. The last line of your response should be of the following format: 'Answer: $VALUE' where VALUE is a numerical value.", 'few_shot_demos': None}, prompt_variables: ['input_str', 'few_shot_demos', 'system_prompt']
    )
    (model_client): OpenAIClient()
    (output_processors): ParseIntegerAnswerCompon

In [ ]:
answer = task_pipeline(question, id="1")
print(answer)

GeneratorOutput(id='1', data=8, error=None, usage=CompletionUsage(completion_tokens=77, prompt_tokens=113, total_tokens=190), raw_response='To find the total number of musical instruments you have, you simply need to count the individual instruments you listed. \n\nYou have:\n- Flute\n- Piano\n- Trombone\n- Violin\n- Accordion\n- Clarinet\n- Drum\n- Trumpet\n\nCounting each of these instruments, we get a total of 8 musical instruments.\n\nAnswer: 8', metadata=None)


In [ ]:
# set it to train mode
task_pipeline.train()
answer = task_pipeline(question, id="1")
print(answer)
print(f"full_response: {answer.full_response}")

Parameter(name=Generator_output, requires_opt=True, param_type=generator_output (The output of the generator.), role_desc=Output from (llm) Generator, data=To find the total number of musical instruments you have, you simply need to count the individual instruments you listed. 

You have:
- Flute
- Piano
- Trombone
- Violin
- Accordion
- Clarinet
- Drum
- Trumpet

Counting each of these instruments, we get a total of 8 musical instruments.

Answer: 8, predecessors={Parameter(name=To_provide, requires_opt=True, param_type=demos (A few examples to guide the language model.), role_desc=To provide few shot demos to the language model, data=None, predecessors=set(), gradients=[],            raw_response=None, input_args=None, traces={}), Parameter(name=To_give_ta, requires_opt=True, param_type=prompt (Instruction to the language model on task, data, and format.), role_desc=To give task instruction to the language model in the system prompt, data=You will answer a reasoning question. Think s

In [ ]:
!pip install datasets
clear_output()

### Load Datasets

In [ ]:
from adalflow.datasets.big_bench_hard import BigBenchHard
from adalflow.utils.data import subset_dataset

def load_datasets(max_samples: int = None):
    """Load the dataset"""
    train_data = BigBenchHard(split="train")
    val_data = BigBenchHard(split="val")
    test_data = BigBenchHard(split="test")

    # Limit the number of samples
    if max_samples:
        train_data = subset_dataset(train_data, max_samples)
        val_data = subset_dataset(val_data, max_samples)
        test_data = subset_dataset(test_data, max_samples)

    return train_data, val_data, test_data


In [ ]:
# check the datasets

train_data, val_data, test_data = load_datasets(max_samples=2)
print(train_data[0])

Example(id='d3f33ded-170a-4b87-9b0b-987d5fb7b817', question='I have a cauliflower, a stalk of celery, a cabbage, and a garlic. How many vegetables do I have?', answer='4')


### Soft link to AdalFlow default file path

Lets' match the default to the current project, so that you can see the downloaded data and later the checkpoints of the training.

In [ ]:
! ln -s /root/.adalflow /content/adalflow

# go to files then you will see a folder named as adalflow

# 😊 AdalComponent to define everything we need to train

1. We need `backward_engine_model_config`` for ``backward_engine`` to compute gradient.

2. We need ``text_optimizer_model_config`` for the `text optimizer` for propose new prompts.

3. For the demo optimizer, we need a `teacher_model_config` to config a teacher generator, in this case, it is the `llm_counter`. The teacher will share the same prompt with the `llm_counter` but you can use a more advanced model.

In general, we should have all of these parts to use a more advanced model.

## 🧑 Diagnose

Diagnose is more of an evaluation, but with detailed logs so that you can manually inspect the wrong output.

This one shows the minimum config you need to get the `diagnose` work.

In [ ]:
from adalflow.datasets.types import Example
from adalflow.eval.answer_match_acc import AnswerMatchAcc


class ObjectCountAdalComponent(adal.AdalComponent):
    def __init__(self, model_client: adal.ModelClient, model_kwargs: Dict):
        task = ObjectCountTaskPipeline(model_client, model_kwargs)
        eval_fn = AnswerMatchAcc(type="exact_match").compute_single_item
        super().__init__(task=task, eval_fn=eval_fn)

    def prepare_task(self, sample: Example):
        return self.task.call, {"question": sample.question, "id": sample.id}

    def prepare_eval(
        self, sample: Example, y_pred: adal.GeneratorOutput
    ) -> float:
        y_label = -1
        if (y_pred is not None and y_pred.data is not None):  # if y_pred and y_pred.data: might introduce bug when the data is 0
            y_label = y_pred.data
        return self.eval_fn, {"y": y_label, "y_gt": sample.answer}

In [ ]:
def diagnose(
    model_client: adal.ModelClient,
    model_kwargs: Dict,
) -> Dict:

    trainset, valset, testset = load_datasets()
    # use max_samples=10 to test the code

    adal_component = ObjectCountAdalComponent(model_client, model_kwargs)
    trainer = adal.Trainer(adaltask=adal_component)
    trainer.diagnose(dataset=trainset, split="train")
    trainer.diagnose(dataset=valset, split="val")
    trainer.diagnose(dataset=testset, split="test")

In [ ]:
diagnose(**gpt_3_model)

cache_path: /root/.adalflow/cache_OpenAIClient_gpt-3.5-turbo.db
Checkpoint path: /root/.adalflow/ckpt/ObjectCountAdalComponent
Save diagnose to /root/.adalflow/ckpt/ObjectCountAdalComponent/diagnose_train
Saving traces to /root/.adalflow/ckpt/ObjectCountAdalComponent/diagnose_train
all_generators: [('llm_counter', Generator(
  model_kwargs={'model': 'gpt-3.5-turbo', 'max_tokens': 2000, 'temperature': 0.0, 'top_p': 0.99, 'frequency_penalty': 0, 'presence_penalty': 0, 'stop': None}, 
  (prompt): Prompt(
    template: <START_OF_SYSTEM_PROMPT>
    {{system_prompt}}
    {# Few shot demos #}
    {% if few_shot_demos is not none %}
    Here are some examples:
    {{few_shot_demos}}
    {% endif %}
    <END_OF_SYSTEM_PROMPT>
    <START_OF_USER>
    {{input_str}}
    <END_OF_USER>
    , prompt_kwargs: {'system_prompt': "You will answer a reasoning question. Think step by step. The last line of your response should be of the following format: 'Answer: $VALUE' where VALUE is a numerical value.", 

Loading Data: 100%|██████████| 50/50 [00:00<00:00, 5957.82it/s]
Evaluating step(0): 0.88 across 50 samples, Max potential: 0.88: 100%|██████████| 50/50 [00:15<00:00,  3.27it/s]


sorted_indices: [8, 16, 23, 25, 31, 47, 0, 1, 2, 3, 4, 5, 6, 7, 9, 10, 11, 12, 13, 14, 15, 17, 18, 19, 20, 21, 22, 24, 26, 27, 28, 29, 30, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 48, 49]
sorted_scores: [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]
Loading log file: llm_counter_call.jsonl
Total error samples: 6
Save diagnose to /root/.adalflow/ckpt/ObjectCountAdalComponent/diagnose_val
Saving traces to /root/.adalflow/ckpt/ObjectCountAdalComponent/diagnose_val
all_generators: [('llm_counter', Generator(
  model_kwargs={'model': 'gpt-3.5-turbo', 'max_tokens': 2000, 'temperature': 0.0, 'top_p': 0.99, 'frequency_penalty': 0, 'presence_penalty': 0, 'stop': None}, 
  (prompt): Prompt(
    template: <START_OF_SYSTEM_PROMPT>
    {{system_prompt}}
    {# Few shot demos #}
 

Loading Data: 100%|██████████| 50/50 [00:00<00:00, 3203.76it/s]
Evaluating step(0): 0.8 across 50 samples, Max potential: 0.8: 100%|██████████| 50/50 [00:15<00:00,  3.26it/s]


sorted_indices: [1, 2, 5, 10, 24, 36, 38, 42, 44, 47, 0, 3, 4, 6, 7, 8, 9, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 37, 39, 40, 41, 43, 45, 46, 48, 49]
sorted_scores: [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]
Loading log file: llm_counter_call.jsonl
Total error samples: 10
Save diagnose to /root/.adalflow/ckpt/ObjectCountAdalComponent/diagnose_test
Saving traces to /root/.adalflow/ckpt/ObjectCountAdalComponent/diagnose_test
all_generators: [('llm_counter', Generator(
  model_kwargs={'model': 'gpt-3.5-turbo', 'max_tokens': 2000, 'temperature': 0.0, 'top_p': 0.99, 'frequency_penalty': 0, 'presence_penalty': 0, 'stop': None}, 
  (prompt): Prompt(
    template: <START_OF_SYSTEM_PROMPT>
    {{system_prompt}}
    {# Few shot demos #

Loading Data: 100%|██████████| 100/100 [00:00<00:00, 5545.09it/s]
Evaluating step(0): 0.83 across 100 samples, Max potential: 0.83: 100%|██████████| 100/100 [00:28<00:00,  3.50it/s]

sorted_indices: [7, 18, 19, 20, 23, 24, 25, 43, 58, 59, 63, 74, 75, 79, 85, 97, 99, 0, 1, 2, 3, 4, 5, 6, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 21, 22, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 60, 61, 62, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 76, 77, 78, 80, 81, 82, 83, 84, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 98]
sorted_scores: [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]
Loading log file: llm_counter_call.jsonl
Total error samples: 17


Now, you can go to `/content/adalflow/ckpt/ObjectCountAdalComponent/diagnose_train/stats.json` to view the average score for each split. And also the `diagnose.json` for different errors.

Here is the overall score for each split.

| Train  | Val| Test |
|:--------- |:--------:| ---------:|
| 0.88      | 0.8   |    0.83  |



## 🐛 Debug

## ✅ Train

Now, let's start training.

In [ ]:
from adalflow.datasets.types import Example
from adalflow.eval.answer_match_acc import AnswerMatchAcc


class ObjectCountAdalComponent(adal.AdalComponent):
    def __init__(
        self,
        model_client: adal.ModelClient,
        model_kwargs: Dict,
        backward_engine_model_config: Dict,
        teacher_model_config: Dict,
        text_optimizer_model_config: Dict,
    ):
        task = ObjectCountTaskPipeline(model_client, model_kwargs)
        eval_fn = AnswerMatchAcc(type="exact_match").compute_single_item
        loss_fn = adal.EvalFnToTextLoss(
            eval_fn=eval_fn,
            eval_fn_desc="exact_match: 1 if str(y) == str(y_gt) else 0",
        )
        super().__init__(task=task, eval_fn=eval_fn, loss_fn=loss_fn)

        self.backward_engine_model_config = backward_engine_model_config
        self.teacher_model_config = teacher_model_config
        self.text_optimizer_model_config = text_optimizer_model_config

    def prepare_task(self, sample: Example):
        return self.task.call, {"question": sample.question, "id": sample.id}

    def prepare_eval(
        self, sample: Example, y_pred: adal.GeneratorOutput
    ) -> float:
        y_label = -1
        if y_pred and y_pred.data:
            y_label = y_pred.data
        return self.eval_fn(y=y_label, y_gt=sample.answer)

    def prepare_loss(self, sample: Example, pred: adal.Parameter):
        # prepare gt parameter
        y_gt = adal.Parameter(
            name="y_gt",
            data=sample.answer,
            eval_input=sample.answer,
            requires_opt=False,
        )

        # pred's full_response is the output of the task pipeline which is GeneratorOutput
        pred.eval_input = pred.full_response.data
        return self.loss_fn, {"kwargs": {"y": pred, "y_gt": y_gt}}

In [ ]:
def train(
    train_batch_size=4,  # larger batch size is not that effective, probably because of llm's lost in the middle
    raw_shots: int = 0,
    bootstrap_shots: int = 1,
    max_steps=1,
    num_workers=4,
    strategy="random",
    optimization_order="sequential",
    debug=False,
    resume_from_ckpt=None,
    exclude_input_fields_from_bootstrap_demos=False,
):
    adal_component = ObjectCountAdalComponent(
        **gpt_3_model,
        teacher_model_config=gpt_4o_model,
        text_optimizer_model_config=gpt_4o_model,
        backward_engine_model_config=gpt_4o_model
    )
    print(adal_component)
    trainer = adal.Trainer(
        train_batch_size=train_batch_size,
        adaltask=adal_component,
        strategy=strategy,
        max_steps=max_steps,
        num_workers=num_workers,
        raw_shots=raw_shots,
        bootstrap_shots=bootstrap_shots,
        debug=debug,
        weighted_sampling=True,
        optimization_order=optimization_order,
        exclude_input_fields_from_bootstrap_demos=exclude_input_fields_from_bootstrap_demos,
    )
    print(trainer)

    train_dataset, val_dataset, test_dataset = load_datasets()
    trainer.fit(
        train_dataset=train_dataset,
        val_dataset=val_dataset,
        test_dataset=test_dataset,
        debug=debug,
        resume_from_ckpt=resume_from_ckpt,
    )


We use `Sequential` in default, we will end up with 24 steps in total, 12 for text optimizer and 12 for the demo optimizer.

In [ ]:
train(debug=False, max_steps=12, strategy="constrained",
      raw_shots=0, bootstrap_shots=1,
      exclude_input_fields_from_bootstrap_demos=True
      )

cache_path: /root/.adalflow/cache_OpenAIClient_gpt-3.5-turbo.db
ObjectCountAdalComponent(
  eval_fn: compute_single_item, backward_engine: None, backward_engine_model_config: {'model_client': OpenAIClient(), 'model_kwargs': {'model': 'gpt-4o', 'max_tokens': 4000, 'temperature': 0.0, 'top_p': 0.99, 'frequency_penalty': 0, 'presence_penalty': 0, 'stop': None}}, teacher_model_config: {'model_client': OpenAIClient(), 'model_kwargs': {'model': 'gpt-4o', 'max_tokens': 4000, 'temperature': 0.0, 'top_p': 0.99, 'frequency_penalty': 0, 'presence_penalty': 0, 'stop': None}}, text_optimizer_model_config: {'model_client': OpenAIClient(), 'model_kwargs': {'model': 'gpt-4o', 'max_tokens': 4000, 'temperature': 0.0, 'top_p': 0.99, 'frequency_penalty': 0, 'presence_penalty': 0, 'stop': None}}
  (task): ObjectCountTaskPipeline(
    (llm_counter): Generator(
      model_kwargs={'model': 'gpt-3.5-turbo', 'max_tokens': 2000, 'temperature': 0.0, 'top_p': 0.99, 'frequency_penalty': 0, 'presence_penalty': 0, '

Loading Data: 100%|██████████| 50/50 [00:00<00:00, 6482.70it/s]
Evaluating step(0): 0.8 across 50 samples, Max potential: 0.8: 100%|██████████| 50/50 [00:00<00:00, 347.01it/s]
Loading Data: 100%|██████████| 100/100 [00:00<00:00, 2017.67it/s]
Evaluating step(0): 0.83 across 100 samples, Max potential: 0.83: 100%|██████████| 100/100 [00:00<00:00, 286.59it/s]


Initial validation score: 0.8
Initial test score: 0.83
Checkpoint path: /root/.adalflow/ckpt/ObjectCountAdalComponent
save to /root/.adalflow/ckpt/ObjectCountAdalComponent/constrained_max_steps_12_4e8a1_run_1.json


Training: 100%|██████████| 4/4 [00:00<00:00, 1489.32it/s]

Training Step: 2:   8%|▊         | 1/13 [00:00<00:01,  8.29it/s]

Skipping batch 0 as acc: 1.0
No proposal can improve the subset and full set, go to next step


...

Trace with id d71ad721-d21d-42f1-af9b-719ff026406b already exists. Updating the trace.



Calculating Loss: 100%|██████████| 4/4 [00:00<00:00, 1513.37it/s]

Training: 100%|██████████| 4/4 [00:01<00:00,  2.07it/s]

Calculating Loss: 100%|██████████| 4/4 [00:00<00:00, 862.32it/s]


sampled_augmented_demos: ['840d9ed5-8222-45a9-a406-7445feae9733']
New prompts: [PromptData(id='44f6083f-4cf7-4a9a-bf10-20d218ee4106', name='llm_counter.system_prompt', data="You will answer a reasoning question. Carefully count each item, paying special attention to quantities mentioned. Verify your total. Think step by step. The last line of your response should be of the following format: 'Answer: $VALUE' where VALUE is a numerical value.", requires_opt=True), PromptData(id='2f948c14-7f8f-4f46-9e23-d30598d3f47b', name='llm_counter.few_shot_demos', data="Example: 'Let''s count each item step by step:\n\n\n  1. Microwave: 1\n\n  2. Table: 1\n\n  3. Fridge: 1\n\n  4. Stove: 1\n\n  5. Oven: 1\n\n  6. Toaster: 1\n\n  7. Couch: 1\n\n  8. Cars: 4\n\n\n  Now, add them up:\n\n\n  1 + 1 + 1 + 1 + 1 + 1 + 1 + 4 = 11\n\n\n  Answer: 11'", requires_opt=True)]



Loading Data: 100%|██████████| 50/50 [00:00<00:00, 1210.01it/s]
Evaluating step(24): 0.86 across 50 samples, Max potential: 0.86: 100%|██████████| 50/50 [00:18<00:00,  2.69it/s]
Training Step: 24: 100%|██████████| 12/12 [02:15<00:00, 11.26s/it]

Fail validation: 0.86 <= 0.86, revert
Saved ckpt to /root/.adalflow/ckpt/ObjectCountAdalComponent/constrained_max_steps_12_4e8a1_run_1.json
Training time: 359.32386112213135s
ckpt_file: /root/.adalflow/ckpt/ObjectCountAdalComponent/constrained_max_steps_12_4e8a1_run_1.json


Here is our scores for each step:

"val_scores": [
        0.8,
        0.8,
        0.8,
        0.8,
        0.8,
        0.84,
        0.84,
        0.84,
        0.84,
        0.86,
        0.86,
        0.86,
        0.86,
        0.86,
        0.86,
        0.86,
        0.86,
        0.86,
        0.86,
        0.86,
        0.86,
        0.86,
        0.86,
        0.86,
        0.86
    ]

  "test_scores": [
        0.83,
        0.83,
        0.83,
        0.83,
        0.83,
        0.91,
        0.91,
        0.91,
        0.91,
        0.87,
        0.87,
        0.87,
        0.87,
        0.87,
        0.87,
        0.87,
        0.87,
        0.87,
        0.87,
        0.87,
        0.87,
        0.87,
        0.87,
        0.87,
        0.87
    ]


It is normal when the score of the validation does not exactly match to that of the test set. You can also train with just the test set. You can modify the fit arguments as

```
trainer.fit(
        train_dataset=train_dataset,
        val_dataset=test_dataset,
        # test_dataset=test_dataset,
        debug=debug,
        resume_from_ckpt=resume_from_ckpt,
    )
```

# 🔥 Resume Checkpoint

We might want to continue from the earlier step and to train more steps

This is easy to do.

**Note: Ensure you copy the path you had, and replace it, as your run might create a different file name.**

In [ ]:

ckpt_path = "/content/adalflow/ckpt/ObjectCountAdalComponent/constrained_max_steps_12_4e8a1_run_1.json"

train(debug=False, max_steps=12, strategy="constrained",
                raw_shots=0, bootstrap_shots=1,
                resume_from_ckpt=ckpt_path,
                exclude_input_fields_from_bootstrap_demos=True)

cache_path: /root/.adalflow/cache_OpenAIClient_gpt-3.5-turbo.db
ObjectCountAdalComponent(
  eval_fn: compute_single_item, backward_engine: None, backward_engine_model_config: {'model_client': OpenAIClient(), 'model_kwargs': {'model': 'gpt-4o', 'max_tokens': 4000, 'temperature': 0.0, 'top_p': 0.99, 'frequency_penalty': 0, 'presence_penalty': 0, 'stop': None}}, teacher_model_config: {'model_client': OpenAIClient(), 'model_kwargs': {'model': 'gpt-4o', 'max_tokens': 4000, 'temperature': 0.0, 'top_p': 0.99, 'frequency_penalty': 0, 'presence_penalty': 0, 'stop': None}}, text_optimizer_model_config: {'model_client': OpenAIClient(), 'model_kwargs': {'model': 'gpt-4o', 'max_tokens': 4000, 'temperature': 0.0, 'top_p': 0.99, 'frequency_penalty': 0, 'presence_penalty': 0, 'stop': None}}
  (task): ObjectCountTaskPipeline(
    (llm_counter): Generator(
      model_kwargs={'model': 'gpt-3.5-turbo', 'max_tokens': 2000, 'temperature': 0.0, 'top_p': 0.99, 'frequency_penalty': 0, 'presence_penalty': 0, '

Training: 100%|██████████| 4/4 [00:00<00:00, 1073.40it/s]

Training Step: 28:   0%|          | 0/13 [00:00<?, ?it/s]

Skipping batch 0 as acc: 1.0
No proposal can improve the subset and full set, go to next step



Training: 100%|██████████| 4/4 [00:02<00:00,  1.83it/s]

Training Step: 29:  15%|█▌        | 2/13 [00:02<00:12,  1.15s/it]

Skipping batch 1 as acc: 1.0
No proposal can improve the subset and full set, go to next step


...

sampled_augmented_demos: ['fd34672a-ffd1-498e-a88f-283aa9d4f65d']
New prompts: [PromptData(id='a530c025-f25c-4423-b146-215ff73586f4', name='llm_counter.system_prompt', data="You will answer a reasoning question. Carefully count each item and verify your total. Think step by step. The last line of your response should be of the following format: 'Answer: $VALUE' where VALUE is a numerical value.", requires_opt=True), PromptData(id='0b4dc918-1afb-4f03-9193-90ec51a9abab', name='llm_counter.few_shot_demos', data="Example: 'Let''s count each item step by step:\n\n\n  1. Microwave: 1\n\n  2. Lamp: 1\n\n  3. Cars: 4\n\n  4. Stove: 1\n\n  5. Toaster: 1\n\n  6. Bed: 1\n\n\n  Now, add them all together:\n\n\n  1 + 1 + 4 + 1 + 1 + 1 = 9\n\n\n  Answer: 9'", requires_opt=True)]



Loading Data: 100%|██████████| 50/50 [00:00<00:00, 2261.91it/s]
Evaluating step(50): 0.825 across 40 samples, Max potential: 0.86:  80%|████████  | 40/50 [00:00<00:00, 281.78it/s]

Fail validation: 0.8048780487804879 <= 0.86, revert



Training Step: 50: 100%|██████████| 12/12 [01:49<00:00,  9.15s/it]


Saved ckpt to /content/adalflow/ckpt/ObjectCountAdalComponent/constrained_max_steps_12_4e8a1_run_1.json
Training time: 352.5873613357544s
ckpt_file: /content/adalflow/ckpt/ObjectCountAdalComponent/constrained_max_steps_12_4e8a1_run_1.json


I decide to try more, this time, using strategy "random". And in the bootstrap demo, there is one shot, but I ensure I also add the "input" in the demonstration.

In [ ]:

train(debug=False, max_steps=12, strategy="random",
                raw_shots=0, bootstrap_shots=1,
                resume_from_ckpt=ckpt_path,
                exclude_input_fields_from_bootstrap_demos=False)

cache_path: /root/.adalflow/cache_OpenAIClient_gpt-3.5-turbo.db
ObjectCountAdalComponent(
  eval_fn: compute_single_item, backward_engine: None, backward_engine_model_config: {'model_client': OpenAIClient(), 'model_kwargs': {'model': 'gpt-4o', 'max_tokens': 4000, 'temperature': 0.0, 'top_p': 0.99, 'frequency_penalty': 0, 'presence_penalty': 0, 'stop': None}}, teacher_model_config: {'model_client': OpenAIClient(), 'model_kwargs': {'model': 'gpt-4o', 'max_tokens': 4000, 'temperature': 0.0, 'top_p': 0.99, 'frequency_penalty': 0, 'presence_penalty': 0, 'stop': None}}, text_optimizer_model_config: {'model_client': OpenAIClient(), 'model_kwargs': {'model': 'gpt-4o', 'max_tokens': 4000, 'temperature': 0.0, 'top_p': 0.99, 'frequency_penalty': 0, 'presence_penalty': 0, 'stop': None}}
  (task): ObjectCountTaskPipeline(
    (llm_counter): Generator(
      model_kwargs={'model': 'gpt-3.5-turbo', 'max_tokens': 2000, 'temperature': 0.0, 'top_p': 0.99, 'frequency_penalty': 0, 'presence_penalty': 0, '

Training: 100%|██████████| 4/4 [00:00<00:00, 224.54it/s]

Calculating Loss: 100%|██████████| 4/4 [00:00<00:00, 10894.30it/s]

Loss backward...


setting pred name Generator_outputy_pred_2 score to 1.0
setting pred name Generator_outputy_pred_1 score to 1.0
setting pred name Generator_outputy_pred_0 score to 1.0
setting pred name Generator_outputy_pred_3 score to 1.0
Optimizer propose...
New prompts:  [PromptData(id='327b63f0-b532-435a-85d7-6137d4e52c4c', name='llm_counter.system_prompt', data="You will answer a reasoning question. Carefully count each item and verify your total. List each item individually. The last line of your response should be of the following format: 'Answer: $VALUE' where VALUE is a numerical value.", requires_opt=True), PromptData(id='73a3953b-6351-44d8-a36f-7521db346cca', name='llm_counter.few_shot_demos', data=None, requires_opt=True)]



Loading Data: 100%|██████████| 50/50 [00:00<00:00, 2199.38it/s]
Evaluating step(51): 0.86 across 50 samples, Max potential: 0.86: 100%|██████████| 50/50 [00:12<00:00,  3.97it/s]
Training Step: 52:   8%|▊         | 1/13 [00:18<03:38, 18.20s/it]

...

Trace with id b538075d-01af-4b76-b835-9005f3044609 already exists. Updating the trace.



Calculating Loss: 100%|██████████| 4/4 [00:00<00:00, 1086.11it/s]

Training: 100%|██████████| 4/4 [00:02<00:00,  1.50it/s]

Calculating Loss: 100%|██████████| 4/4 [00:00<00:00, 14242.12it/s]


sampled_augmented_demos: ['b538075d-01af-4b76-b835-9005f3044609']
New prompts: [PromptData(id='327b63f0-b532-435a-85d7-6137d4e52c4c', name='llm_counter.system_prompt', data="You will answer a reasoning question. Carefully count each item and verify your total. List each item individually and ensure accuracy. Show your calculations step by step. The last line of your response should be: 'Answer: $VALUE' where VALUE is a numerical value.", requires_opt=True), PromptData(id='73a3953b-6351-44d8-a36f-7521db346cca', name='llm_counter.few_shot_demos', data="input_str: I have a blackberry, a raspberry, a peach, a head of broccoli, a plum,\n  an orange, two bananas, a grape, two garlics, a nectarine, a lettuce head, and an\n  apple. How many fruits do I have?\nExample: 'Let''s list each item and identify whether it is a fruit:\n\n\n  1. Blackberry - Fruit\n\n  2. Raspberry - Fruit\n\n  3. Peach - Fruit\n\n  4. Head of broccoli - Not a fruit\n\n  5. Plum - Fruit\n\n  6. Orange - Fruit\n\n  7. Tw


Loading Data: 100%|██████████| 50/50 [00:00<00:00, 634.93it/s]
Evaluating step(74): 0.5 across 4 samples, Max potential: 0.96:   8%|▊         | 4/50 [00:28<05:25,  7.07s/it]
Training Step: 74: 100%|██████████| 12/12 [03:12<00:00, 16.04s/it]

Fail validation: 0.4 <= 0.96, revert
Saved ckpt to /content/adalflow/ckpt/ObjectCountAdalComponent/constrained_max_steps_12_4e8a1_run_1.json
Training time: 484.17421078681946s
ckpt_file: /content/adalflow/ckpt/ObjectCountAdalComponent/constrained_max_steps_12_4e8a1_run_1.json


Finally, we got 96% on the val and 95% on the test!!! This is really close to GPT4o's performance. This took us 72 steps!

The score is consistent, meaning this is a good prompt.
Here is our final optimized prompt:

System:

```

"prompt": [
                {
                    "id": "327b63f0-b532-435a-85d7-6137d4e52c4c",
                    "name": "llm_counter.system_prompt",
                    "data": "You will answer a reasoning question. Carefully count each item and verify your total. List each item individually and ensure accuracy. Show your calculations step by step. The last line of your response should be: 'Answer: $VALUE' where VALUE is a numerical value.",
                    "requires_opt": true
                },
                {
                    "id": "73a3953b-6351-44d8-a36f-7521db346cca",
                    "name": "llm_counter.few_shot_demos",
                    "data": "input_str: I have a yam, a cauliflower, a bed, two cabbages, a garlic, an oven, a\n  carrot, a head of broccoli, a potato, a stalk of celery, a lettuce head, and a toaster.\n  How many vegetables do I have?\nExample: 'Let''s list and count each vegetable individually:\n\n\n  1. Yam\n\n  2. Cauliflower\n\n  3. Cabbage (1)\n\n  4. Cabbage (2)\n\n  5. Garlic\n\n  6. Carrot\n\n  7. Broccoli\n\n  8. Potato\n\n  9. Celery\n\n  10. Lettuce\n\n\n  Now, let''s verify the count:\n\n\n  1. Yam\n\n  2. Cauliflower\n\n  3. Cabbage (1)\n\n  4. Cabbage (2)\n\n  5. Garlic\n\n  6. Carrot\n\n  7. Broccoli\n\n  8. Potato\n\n  9. Celery\n\n  10. Lettuce\n\n\n  Total number of vegetables: 10\n\n\n  Answer: 10'",
                    "requires_opt": true
                }
            ]
```


You will see all steps record from the log.

Happy Optimizing!!!

# Issues and feedback

If you encounter any issues, please report them here: [GitHub Issues](https://github.com/SylphAI-Inc/LightRAG/issues).

For feedback, you can use either the [GitHub discussions](https://github.com/SylphAI-Inc/LightRAG/discussions) or [Discord](https://discord.gg/ezzszrRZvT).